In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
import csv


In [3]:
dat = pd.read_csv("E:\MATH5470\data\data_clean.csv")

In [4]:
dat["sic2"] = dat["sic2"].fillna(value=0)
sic = pd.get_dummies(dat["sic2"], prefix = "sic")
sic = sic.iloc[:,1:]
dat = dat.drop("sic2", axis = 1)
dat = pd.concat([dat,sic],axis = 1)
# sic2, sic2_counts = np.unique(dat["sic2"], return_counts=True)
# def map_sic2(x):
#     return np.where(sic2 == x)[0][0]
# dat["sic2"] = list(map(map_sic2, dat["sic2"]))
# sic2_onehot = pd.get_dummies(dat["sic2"], prefix='sic2')
# sic2_names = sic2_onehot.columns
# data = pd.concat([dat, sic2_onehot], axis=1)
# dat = pd.concat([dat, sic], axis = 1)

column_names = dat.columns


In [38]:
pred_names = dat.iloc[:,4:].columns
pred_names

Index(['mvel1', 'beta', 'betasq', 'chmom', 'dolvol', 'idiovol', 'indmom',
       'mom1m', 'mom6m', 'mom12m',
       ...
       'sic_79.0', 'sic_80.0', 'sic_81.0', 'sic_82.0', 'sic_83.0', 'sic_84.0',
       'sic_86.0', 'sic_87.0', 'sic_89.0', 'sic_99.0'],
      dtype='object', length=176)

In [6]:
def r2_oos(y_predict, y_test):
    result = 1 - (np.sum((y_predict - y_test)**2)/np.sum(y_test**2))
    return result

In [7]:
# def compute_variable_importance(model, x_train, y_train, names):
#     y_predict = model.predict(x_train)
#     roos_all = r2_oos(y_predict, y_train)
#     vi_array = np.array([])
#     for i in names:
#         x_temp = x_train.copy(deep=True)
#         x_temp[i] = 0
#         y_predict = model.predict(x_temp)
#         roos_temp = r2_oos(y_predict, y_train)
#         vi_array = np.append(vi_array, roos_all-roos_temp)
#     return vi_array

In [71]:
test_pred = []

for i in range(30):
    
  # x_train = dat[dat['date'] < 197500 + i * 100][pred_names]
  # y_train = dat[dat['date'] < 197500 + i * 100]['RET']
  # x_valid = dat[(dat['date'] >= 197500 + i * 100) & (dat['date'] < 198700 + i * 100)][pred_names]
  # y_valid = dat[(dat['date'] >= 197500 + i * 100) & (dat['date'] < 198700 + i * 100)]['RET']

  x_train = dat[dat['date'] < 198700 + i * 100][pred_names]
  y_train = dat[dat['date'] < 198700 + i * 100]['RET']
  x_test = dat[(dat['date'] >= 198700 + i * 100) & (dat['date'] < 198800 + i * 100)][pred_names]
  y_test = dat[(dat['date'] >= 198700 + i * 100) & (dat['date'] < 198800 + i * 100)]['RET']
    
  # train model (tuning parameter)
    
  # r2_oos_valid = 1
  # best_depth = 0
  # importance = []
  # for j in range (1,4):
  #   regr = RandomForestRegressor(n_estimators=50,max_features=30, max_depth=j)
  #   regr.fit(x_train, y_train)
  #   temp_pred = regr.predict(x_valid)
  #   if (abs(r2_oos(temp_pred,y_valid)) < abs(r2_oos_valid)):
  #       r2_oos_valid = r2_oos(temp_pred,y_valid)
  #       best_depth = j
  
  # best_regr = RandomForestRegressor(n_estimators=50,max_features=30, max_depth=best_depth)
  # best_regr.fit(x_train, y_train)

  # importance.append (best_regr.feature_importances_*10)
  # test_pred.append(best_regr.predict(x_test))


  regr = RandomForestRegressor(n_estimators=50, max_features=30, max_depth=4)
  regr.fit(x_train,y_train)

  test_pred += regr.predict(x_test).tolist()
  print(i)


    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [72]:
result = pd.DataFrame({'prediction':test_pred})
result.to_csv("rf_pred.csv")
# import joblib
# joblib.dump(regr,"rf_fit.joblib")

In [73]:
dat_test = dat[(dat['date'] >= 198700)]['RET'].reset_index()
rf_oos = pd.concat([dat_test,result],axis=1)
r2_oos(rf_oos['prediction'],rf_oos['RET'])

In [57]:
var_names = pred_names[0:102]
r2_full = r2_oos(regr.predict(x_train).tolist(),y_train)
vi = []
for var in var_names:
    x_restricted = x_train.copy()
    x_restricted[var] = 0
    r2_restricted = r2_oos(regr.predict(x_restricted).tolist(),y_train)
    vi.append(r2_full-r2_restricted)
vi_result = pd.DataFrame({'variable':var_names,'VI':vi})
vi_result['VI'] = vi_result['VI']/sum(vi_result['VI'])


mvel1
beta
betasq
chmom
dolvol
idiovol
indmom
mom1m
mom6m
mom12m
mom36m
pricedelay
turn
absacc
acc
age
agr
cashdebt
cashpr
cfp
cfp_ia
chatoia
chcsho
chempia
chinv
chpmia
convind
currat
depr
divi
divo
dy
egr
ep.x
gma
grcapx
grltnoa
herf
hire
invest
lev
lgr
mve_ia
operprof
orgcap
pchcapx_ia
pchcurrat
pchdepr
pchgm_pchsale
pchquick
pchsale_pchinvt
pchsale_pchrect
pchsale_pchxsga
pchsaleinv
pctacc
ps
quick
rd
rd_mve
rd_sale
realestate
roic
salecash
saleinv
salerec
secured
securedind
sgr
sin
sp
tang
tb
aeavol
cash
chtx
cinvest
ear
nincr
roaq
roavol
roeq
rsup
stdacc
stdcf
ms
baspread
ill
maxret
retvol
std_dolvol
std_turn
zerotrade
bm.x
bm_ia
dp
ep.y
bm.y
ntis
tbl
tms
dfy
svar


In [67]:
vi_result = vi_result.sort_values(by = ['VI'],ascending=False)
vi_result

variable        VI
0         mvel1  0.589397
101        svar  0.094279
94           dp  0.050465
95         ep.y  0.044324
100         dfy  0.040263
..          ...       ...
60   realestate -0.001068
8         mom6m -0.001150
9        mom12m -0.002071
34          gma -0.002459
37         herf -0.014004

[102 rows x 2 columns]

In [68]:
vi_result.to_csv("rf_vi.csv")